<a href="https://colab.research.google.com/github/gtradigo/InnoprostProteomicPipeline/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline for clinical and proteomic data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from scipy.stats import reciprocal, uniform
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
from sklearn.decomposition import PCA
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']= 18, 8

import warnings
warnings.filterwarnings("ignore")

np.random.seed(123456)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import the biological dataset containing peptide data. Data will be normalized and analyzed through the pipeline phases.


In [ ]:
# load dataset
data = pd.read_excel('/content/drive/...', sheet_name = '...')
# delete unnamed columns
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
# data preview
data

## Data preprocessing




In [ ]:
# print column names
data.columns

In [ ]:
# delete unwanted columns
data = data.drop(['Name','Gleason'], axis=1)

Optionally impute missing values



In [ ]:
# replace not a number with zeroes
data = data.replace(np.nan,0.0)
# replace infinitive values with zeroes
data = data.replace(np.inf,0.0)
# report missing values
data.isna().sum()

Optionally delete row not having the class variable

In [ ]:
class_column = 'COLUMN COINTAING CLASS'
# delete rows without target value
data = data.drop(data[data[class_column] == 0.0].index, axis=0)
data = data.reset_index()
data = data.drop(['index'], axis = 1)
data.head()

Now uniform back all zeroes to "not a number" values before value imputation

In [ ]:
data = data.replace(0.0, np.nan)
data.isna().sum()

## Missing values imputation

In [ ]:
for column in data:
  data[column] = data[column].fillna(data[column].mean())

In [ ]:
# count how many rows are in each target
data[class_column].value_counts()

Let's binarize the class variable: 0 for the first one, 1 for the second one

In [ ]:
data[class_column].loc[data[class_column] == 'FIRST_CLASS_LABEL' ] = 0
data[class_column].loc[data[class_column] == 'SECOND_CLASS_LABEL' ] = 1

In [ ]:
# output variable
Y = data[class_column]
# training data
X_1 = data.drop([class_column], axis=1)

# Feature selection

Select a subset of variables significant for the analysis phases, by using several (and optional) Machine Learning tools:

* **Pearson correlation**
* **Chi-square test**
* **RFE**
* **Logistic regression**
* **Random Forest**




In [ ]:
# convert output to float type
Y = Y.astype(float)
# max number of features
num_feats = 20

#  Pearson correlation


In [ ]:
X_scaled = MinMaxScaler().fit_transform(X_1)
X_scaled = pd.DataFrame(X_scaled, columns = X_1.columns)

In [ ]:
def cor_selector(X, Y, num_feats):
  cor_list = []
  feature_name = X.columns.tolist()

  for i in X.columns.tolist():
    cor = np.corrcoef(X[i], Y)[0, 1]
    cor_list.append(cor)
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    cor_feature = X.iloc[:, np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    cor_support = [True if i in cor_feature else False for i in feature_name]

  return cor_support, cor_feature

cor_support, cor_feature = cor_selector(X_scaled, Y, num_feats)
print(str(len(cor_feature)), 'selected features')
print(cor_feature)

# Chi square


In [ ]:
chi_selector = SelectKBest(chi2, k = num_feats)
chi_selector.fit(X_scaled, Y)
chi_support = chi_selector.get_support()
chi_feature = X_scaled.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')
print(chi_feature)

# RFE - Recursive Feature Elimination


In [ ]:
rfe_selector = RFE(estimator = LogisticRegression(), n_features_to_select = num_feats, step = 5, verbose = 5)
rfe_selector.fit(X_scaled, Y)
rfe_support = rfe_selector.get_support()
rfe_feature = X_scaled.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
print(rfe_feature)

# Logistic Regression


In [ ]:
embedded_lr_selector = SelectFromModel(LogisticRegression(penalty = "l1", solver = 'liblinear'), max_features = num_feats)
embedded_lr_selector.fit(X_scaled, Y)
embedded_lr_support = embedded_lr_selector.get_support()
embedded_lr_feature = X_scaled.loc[:,embedded_lr_support].columns.tolist()
print(str(len(embedded_lr_feature)), 'selected features')
print(embedded_lr_feature)

# Random Forest



In [ ]:
embedded_rf_selector = SelectFromModel(RandomForestClassifier(30), max_features = num_feats)
embedded_rf_selector.fit(X_scaled, Y)
embedded_rf_support = embedded_rf_selector.get_support()
embedded_rf_feature = X_scaled.loc[:,embedded_rf_support].columns.tolist()
print(str(len(embedded_rf_feature)), 'selected features')
print(embedded_rf_feature)

# Feature selection table


In [ ]:
feature_name= X_scaled.columns

feature_selection_df = pd.DataFrame({'Feature': feature_name,
                                     'Pearson': cor_support,
                                     'Chi-2': chi_support,
                                     'RFE': rfe_support,
                                     'Logistic regression': embedded_lr_support,
                                     'Random Forest': embedded_rf_support})

feature_selection_df['Total'] = np.sum(feature_selection_df, axis = 1)
feature_selection_df = feature_selection_df.sort_values(['Total', 'Feature'], ascending = False)
feature_selection_df.index = range(1, len(feature_selection_df) + 1)
feature_selection_df[:20]

Now select the columns on which at least 'min_score' feature selection tools agreed to be relevant


In [ ]:
min_score = 4

features = []
for row in feature_selection_df.itertuples():
  if (row[-1] >= min_score):
    features.append(row[1])

features

# Machine Learning models training

We will split training and test sets by using a threshold (typically 0.3 for test, 0.7 for training data)

## Training and test set splitting

In [ ]:
train_set, test_set = train_test_split(data, test_size = 0.3, random_state = 42)

y_train = train_set[class_column]
x_train = train_set[features]

y_test = test_set[class_column]
x_test = test_set[features]
print(len(train_set), "train, ", len(test_set), "test")

# ML Training

## Random Forest Classifier



In [ ]:
# this vector will store all of the models' accuracies
accuracies = []

In [ ]:
rf_fit = RandomForestClassifier(n_estimators = 8, criterion = "gini", min_samples_split = 2, bootstrap = True,
                                max_features = 'auto', random_state = 42, min_samples_leaf = 1)
rf = rf_fit.fit(x_train, y_train)
rf_fit_pred = rf.predict(x_test)
acc = accuracy_score(y_test, rf_fit_pred)
accuracies.append(acc)

In [ ]:
# optionally print the confusion matrix
print ("Random Forest - Test Confusion Matrix\n\n", pd.crosstab(y_test, rf_fit_pred, rownames = ["Actual"],colnames = ["Predicted"]))
print ("\nRandom Forest - Test accuracy", round(acc, 3))
print ("\nRandom Forest - Test Classification Report\n", classification_report(y_test, rf_fit_pred))

## Logistic Regression





In [ ]:
lr = LogisticRegression()
lr = lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)
acc = accuracy_score(y_test, lr_pred)
accuracies.append(acc)

In [ ]:
# optionally print the confusion matrix
print ("Logistic Regression - Test Confusion Matrix\n\n", pd.crosstab(y_test, lr_pred, rownames = ["Actual"],colnames = ["Predicted"]))
print ("\n Test accuracy", round(acc, 3))
print ("\n Test Classification Report\n", classification_report(y_test, lr_pred))

## KNN K-Nearest Neighbors

In [ ]:
def best_k(k):

    best_acc = 0.0
    K = None

    for i in k:
        knn_fit = KNeighborsClassifier(n_neighbors = i)
        knn = knn_fit.fit(x_train, y_train)
        knn_pred = knn1.predict(x_test)
        acc = round(accuracy_score(y_test, knn_pred), 3)
        if acc > best_acc:
            K = i
            print('Best K:', K, pd.crosstab(y_test, knn_pred, rownames = ["Actual"],colnames = ["Predicted"]))
            best_acc = acc
            print('\nAccuracy:', best_acc)
            print(classification_report(y_test, knn_pred))

    best_model_knn = KNeighborsClassifier(n_neighbors = K)

    accuracies.append(best_acc)
    return best_model_knn

k = [1,3]
best_model_knn = best_k(k)

## SVM - Support Vector Machine

In [ ]:
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVC(), param_distributions, n_iter = 10, verbose = 2, cv = None, random_state = 42)
rnd_search_cv = rnd_search_cv.fit(x_train, y_train)

In [ ]:
svclassifier = SVC(C = 1.2058449429580245, gamma = 0.0870602087830485, probability = True)
svc = svclassifier.fit(x_train, y_train)
y_pred = svc.predict(x_test)
acc = accuracy_score(y_test, y_pred)
accuracies.append(acc)

In [ ]:
# optionally print confusion matrix
print ("SVM - Test Confusion Matrix\n\n", pd.crosstab(y_test, y_pred, rownames = ["Actual"], colnames = ["Predicted"]))
print ("\nSVM - Test accuracy", round(acc, 3))
print ("\nSVM - Test Classification Report\n", classification_report(y_test, y_pred))

## Decision Tree Classifier

In [ ]:
dtree = DecisionTreeClassifier()
dtc = dtree.fit(x_train, y_train)
y_pred_dtree = dtc.predict(x_test)
acc = accuracy_score(y_test, y_pred_dtree)
accuracies.append(acc)

In [ ]:
# optionally print confusion matrix
print ("Decision Tree - Test Confusion Matrix\n\n", pd.crosstab(y_test, y_pred_dtree, rownames = ["Actual"],colnames = ["Predicted"]))
print ("\nDecision Tree - Test accuracy", round(acc, 3))
print ("\nDecision Tree - Test Classification Report\n", classification_report(y_test, y_pred_dtree))

## Results


In [ ]:
models = {"Random Forest" : rf_fit,
          "Logistic Regression" : lr,
          "KNN" : best_model_knn,
          "SVM" : svclassifier,
          "Decision Tree" : dtree}

# ROC curve

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]

# Instantiate the classfiers and make a list
classifiers = [rf_fit, lr, best_model_knn, svclassifier, dtree]

# Define a result table as a DataFrame
result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])

# Train the models and record the results
for cls in classifiers:
  model = cls.fit(x_train, y_train)
  yproba = model.predict(x_test)
  fpr, tpr, _ = roc_curve(y_test,  yproba)
  auc = roc_auc_score(y_test, yproba)
  result_table1 = result_table.append({'classifiers':cls.__class__.__name__,
                                        'fpr':fpr,
                                        'tpr':tpr,
                                        'auc':auc}, ignore_index=True)

# Set name of the classifiers as index labels
result_table.set_index('classifiers', inplace=True)

fig = plt.figure(figsize=(8,6))

for i in result_table.index:
    plt.plot(result_table.loc[i]['fpr'],
             result_table.loc[i]['tpr'],
             label="{}, AUC={:.3f}".format(i, result_table.loc[i]['auc']))

plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()

# Voting

We will use soft voting (i.e. weighting ML models by their accuracies) and hard voting (i.e. majority of according models)

In [ ]:
from sklearn.ensemble import VotingClassifier

# define validation dataset
validation_data = ...

In [ ]:
# soft voting
s_voting = VotingClassifier(estimators=[('Rf', rf_fit),
                                        ('Lr', lr),
                                        ('Knn', best_model_knn),
                                        ('Svc', svclassifier),
                                        ('Dtc', dtree)],
                            voting='soft',
                            weights=accuracies)
s_voting = s_voting.fit(data, Y)
soft_voting = s_voting.predict(validation_data)

In [ ]:
# hard voting
h_voting = VotingClassifier( estimators=[ ('Rf', rf_fit),
                                          ('Lr', lr),
                                          ('Knn', best_model_knn),
                                          ('Svc', svclassifier),
                                          ('Dtc', dtree)],
                             voting='hard')
h_voting = h_voting.fit(data, Y)
hard_voting = h_voting.predict(validation_data)

## Voting tables




In [ ]:
tab_voting = pd.concat([X, soft_voting, hard_voting], axis = 1)
tab_voting